In [ ]:
# Import libraries
import numpy as np
import math
import random
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm # linear regression
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scanpy as sc
import os
import ProxseqClasses as PC
import SpatialproxseqClasses as SPC

#*****
mpl.rcdefaults()
# Set font to be arial
mpl.rc('font', **{'sans-serif':'Arial', 'size':12})
mpl.rcParams['mathtext.rm'] = 'sans' 
mpl.rcParams['axes.titlesize'] = 12
# Set default tick size
mpl.rcParams['xtick.major.size'] = 5.5
mpl.rcParams['ytick.major.size'] = 5.5
mpl.rcParams['xtick.minor.size'] = 2.5
mpl.rcParams['ytick.minor.size'] = 2.5
# Default legend settings
mpl.rcParams['legend.fancybox'] = False
mpl.rcParams['legend.edgecolor'] = 'k'

#to store text as text, not as path
new_rc_params = {'text.usetex': False,
                 "svg.fonttype": 'none'}
mpl.rcParams.update(new_rc_params)
#*****
# Seed number
np.random.seed(2025)
random.seed(2025)

In [ ]:
#for spatial_RNA data load, just apply sc.read_visium 
#read_visium data 
adata = sc.read_visium('D1_outs')
adata.var_names_make_unique()
adata.obs['mRNA']= 'RNA'
#mitochodria gene
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
sc.set_figure_params(figsize=(4, 4))


In [ ]:
print(adata.shape)

In [ ]:
sc.pl.spatial(adata, color = ['total_counts', 'n_genes_by_counts'], size = 1.5,save = 'Extended-Fig-2f.svg')

In [ ]:
sc.pl.spatial(adata, color=None, spot_size=0, save = 'Extended-Fig-2e.svg')

In [ ]:
# For RNA data:
mean_counts = np.mean(adata.obs['total_counts'])
median_counts = np.median(adata.obs['total_counts'])
std_counts = np.std(adata.obs['total_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")

# For RNA data:
mean_counts = np.mean(adata.obs['n_genes_by_counts'])
median_counts = np.median(adata.obs['n_genes_by_counts'])
std_counts = np.std(adata.obs['n_genes_by_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")


In [ ]:
#QC_scatter_plot
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#perform UMI based filter
sc.pp.filter_cells(adata, min_counts = 1000)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4) # scale each cell to a common library size
sc.pp.log1p(adata) # log(expression + 1)
adata.raw = adata.copy() # freeze the state in `.raw`
adata.write("adata_D1_rna_raw_data.h5ad")

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=4000,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
sc.pl.highly_variable_genes(adata, log=True)
# subset to highly variable genes
adata = adata[:, adata.var.highly_variable].copy()
sc.pp.scale(adata, max_value=10) #normalize genes
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=15) # compute nearest neighbors, k=10 here and n_pcs=30 according to the variance ratio
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.5, key_added='leiden_mrna') # resolution is customly chosen
sc.pl.spatial(adata, color = ['leiden_mrna'],save = 'Extended-Fig-4d-2.svg')

In [ ]:
sc.pl.umap(adata, color = ['leiden_mrna'],save = 'Extended-Fig-4d-1.svg')

In [ ]:
# Step 1: Subset GC cluster (2 and 3)
adata_gc = adata[adata.obs["leiden_mrna"] .isin(["0", "6",])].copy()

# Step 2: Re-cluster
# restore raw counts
adata_gc.X = adata_gc.layers["counts"].copy()

# store raw counts for dotplot etc.
adata_gc.raw = adata_gc.copy()

# normalization and clustering
sc.pp.normalize_total(adata_gc, target_sum=1e4)
sc.pp.log1p(adata_gc)
sc.pp.highly_variable_genes(adata_gc, n_top_genes=4000)
adata_gc = adata_gc[:, adata_gc.var["highly_variable"]].copy()
sc.pp.scale(adata_gc)
sc.tl.pca(adata_gc)
sc.pp.neighbors(adata_gc)
sc.tl.umap(adata_gc)

In [ ]:
sc.tl.leiden(adata_gc, resolution=0.2)

In [ ]:
sc.pl.umap(adata_gc, color = ['leiden'], size = 30)

In [ ]:
# Step 3: assign back the refined cluster labels to original data
adata.obs.loc[adata.obs["leiden_mrna"].isin(["0", "6"]), "lz_dz_subtype"] = adata_gc.obs["leiden"].values



In [ ]:
mask = adata.obs["leiden_mrna"].isin(["0", "6"])
adata.obs.loc[mask & (adata.obs["lz_dz_subtype"] == "0"), "leiden_mrna"] = "0"
adata.obs.loc[mask & (adata.obs["lz_dz_subtype"] == "1"), "leiden_mrna"] = "6"

In [ ]:
sc.pl.umap(adata, color = ['leiden_mrna'])

In [ ]:
cluster_mapping_mrna = {
    '0': 'Light zone',
    '1': 'Epithelial-basal cell',
    '2': 'T:B border',
    '3': 'Mantle zone',
    '4': 'Epithelial-crypt',
    '5': 'Epithelial-crypt',
    '6': 'Dark zone',
    '7': 'T cell zone',
    '8': 'T cell zone',
    '9': 'Epithelial-crypt',
}
# Create a new column with the combined cluster name
adata.obs['mrna_annotation'] = adata.obs['leiden_mrna'].map(cluster_mapping_mrna).fillna(adata.obs['leiden_mrna'])


In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_mrna', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groupby='leiden_mrna', method='wilcoxon')

In [ ]:
sc.tl.dendrogram(adata, groupby='leiden_mrna', key_added='dendrogram_annotation')

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby='leiden_mrna', swap_axes=True, use_raw=True, 
                                cmap='bwr', figsize=(10, 9))

In [ ]:
sc.pl.umap(adata, color = ['S100A8', 'KRT5', 'SFN', 'S100A2', 'KRT14', 'SPINK5', 'S100A14', 'KRT6A', 'S100A16', 'KRT13', 'mrna_annotation' ], color_map='coolwarm', ) #basal cells

In [ ]:
sc.pl.dotplot(adata, ['S100A8', 'KRT5', 'SFN', 'S100A2', 'KRT14', 'SPINK5', 'S100A14', 'KRT6A', 'S100A16', 'KRT13',], groupby="mrna_annotation") #basal

In [ ]:
sc.pl.umap(adata, color = ['ARHGEF16', 'ERRFI1', 'KAZN', 'EPHA2',   'ARHGEF10L', 'PLA2G2F', 'CAMK2N1', 'mrna_annotation'], vmin  = 0, vmax = 1.1, color_map='coolwarm', ) #Surface epithelium

In [ ]:
sc.pl.dotplot(adata, ['ARHGEF16', 'ERRFI1', 'KAZN', 'EPHA2',   'ARHGEF10L', 'PLA2G2F', 'CAMK2N1',], groupby="mrna_annotation") #Surface epithelium

In [ ]:
sc.pl.umap(adata, color = ['CD3D', 'CD3E', 'CD3G', 'CD2', 'IL7R', 'TCF7', 'CCR7', 'LEF1', 'NOSIP', 'GIMAP7', 'mrna_annotation' ], color_map='coolwarm', ) #CD4 naive

In [ ]:
sc.pl.dotplot(adata, ['CD3D', 'CD3E', 'CD4', 'CD3G', 'CD2', 'IL7R', 'TCF7', 'CCR7', 'LEF1', 'NOSIP', 'GIMAP7',], groupby="mrna_annotation") #CD4 naive

In [ ]:
sc.pl.umap(adata, color = ['FCER2', 'IGHD', 'CD83', 'NFKB2', 'CD72', 'CD69', 'MYC', 'EGR3', 'REL', 'HLA-DQB1', 'mrna_annotation' ], color_map='coolwarm', ) #B active marker

In [ ]:
sc.pl.dotplot(adata, ['FCER2', 'IGHD', 'CD83', 'NFKB2', 'CD72', 'CD69', 'MYC', 'EGR3', 'REL', 'HLA-DQB1','CD27'], groupby="mrna_annotation") #B active marker

In [ ]:
sc.pl.umap(adata, color = ['IGHD', 'IGHM', 'BANK1', 'FCER2', 'TCL1A', 'CD79B', 'MEF2C', 'MS4A1', 'CD79A', 'CD22','CD38', 'mrna_annotation' ], color_map='coolwarm', ) #B naive marker

In [ ]:
sc.pl.dotplot(adata, ['IGHD', 'IGHM', 'BANK1', 'FCER2', 'TCL1A', 'CD79B', 'MEF2C', 'MS4A1', 'CD79A', 'CD22',], groupby="mrna_annotation") #B naive marker

In [ ]:
sc.pl.umap(adata, color = ['CD83','LMO2','BCL2A1','mrna_annotation'],  color_map='coolwarm', ) # light zone marker

In [ ]:
sc.pl.dotplot(adata,['CD83','LMO2','BCL2A1',],  groupby="mrna_annotation") # light zone marker

In [ ]:
sc.pl.dotplot(adata,['PASK','ICOS','TIGIT','IL21','IFITM1'],  groupby="mrna_annotation") # Tfh marker

In [ ]:
sc.pl.dotplot(adata,['FDCSP','CLU','CR2','CXCL13'],  groupby="mrna_annotation") # FDC marker

In [ ]:
sc.pl.umap(adata, color = ['CXCR4','AICDA','MME','FOXP1','mrna_annotation'], color_map='coolwarm', ) #dark zone marker

In [ ]:
sc.pl.dotplot(adata,['CXCR4','AICDA','MME','FOXP1',],  groupby="mrna_annotation") # dark zone marker

In [ ]:
sc.pl.umap(adata, color = ['LYZ', 'CST3', 'LGALS2', 'CPVL', 'FGL2', 'AIF1', 'LST1', 'SAMHD1', 'GSN', 'FCER1G','mrna_annotation'], color_map='coolwarm', ) #DC

In [ ]:
sc.pl.dotplot(adata,['LYZ', 'CST3', 'LGALS2', 'CPVL', 'FGL2', 'AIF1', 'LST1', 'SAMHD1', 'GSN', 'FCER1G',],  groupby="mrna_annotation") #DC

In [ ]:
sc.pl.spatial(adata, color = ['mrna_annotation'], save = 'Extended-Fig-4d-3.svg')

In [ ]:
cluster_df = pd.DataFrame({
    'cell_index': adata.obs.index,
    'leiden_mrna':adata.obs['leiden_mrna'],
    'mrna_annotation': adata.obs['mrna_annotation'],
})
cluster_df.to_csv('D1_adata_mRNA_cell_clusters.csv', index=False)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_mrna', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groupby='leiden_mrna', method='wilcoxon')

In [ ]:
custom_gene_names = [
    'CR1', 'VCAM1', 'IGHM', 'IGHG1', 'CD86', 'PDCD1LG2', 'CR2', 'FCER2', 'IGHD',
    'MS4A1', 'CD19', 'CD40', 'ITGA4', 'ITGB1', 'CD81', 'CD38', 'CD8A', 'CD3E', 
    'CD4', 'CD27', 'PDCD1', 'CD28', 'FCGR2A', 'CD9', 'BSG', 'ICAM1', 'ITGAL', 'CD24'
]


In [ ]:

sc.pl.heatmap(
    adata,
    var_names=custom_gene_names,
    groupby='leiden_mrna',
    swap_axes=True,
    cmap='bwr',
    use_raw=True,
    vmin=-5,
    vmax=5,
    figsize=(10, 8)
)

In [ ]:
#read PLA data and transform the data
data = pd.read_csv('D1-PLA_count_matrix.txt.gz', sep="\t",index_col=0)

In [ ]:
# Update row names (index)
data.index = data.index.str.replace(r"^VCAM1_B:(.*)$", r"CD40_A:\1", regex=True)
data.index = data.index.str.replace(r"^CD29_B:(.*)$", r"CD279_A:\1", regex=True)
data.index = data.index.str.replace(r"^LFA1_B:(.*)$", r"CD11a_A:\1", regex=True)

In [ ]:
valid_condition = ~data.index.str.contains(r'((:.*_A)|(_B:))', regex=True)

# Apply the filter to the dataframe
pla = data[valid_condition]

#remover suffix 
pla.index = pla.index.str.replace(r'(_A|_B)', '', regex=True)

In [ ]:
# #exclude HLADPRQ
pla = pla.loc[~pla.index.str.contains('HLADPRQ'),:]

In [ ]:
pla_obj = SPC.sproxseqObject(pla)
pla_obj.compute_protein_pair_counts()
pla_obj.compute_protein_abundance()


In [ ]:
#read tissue barcode 
tissue_barcode = pd.read_csv('D1_outs/spatial/tissue_positions.csv')
tissue_barcode = tissue_barcode.set_index('barcode')
#keep only PLA spot barcode 
tissue_barcode = tissue_barcode.loc[pla.transpose().index,:]

In [ ]:
#construct pla anndata object
adata_pla = sc.AnnData(pla.transpose().to_numpy(), obsm={'spatial': tissue_barcode[['pxl_col_in_fullres','pxl_row_in_fullres']].to_numpy()}, obs=tissue_barcode[['in_tissue','array_row','array_col']])
adata_protein = sc.AnnData(pla_obj.protein_count.transpose().to_numpy(), obsm={'spatial': tissue_barcode[['pxl_col_in_fullres','pxl_row_in_fullres']].to_numpy()}, obs=tissue_barcode[['in_tissue','array_row','array_col']])
adata_protein_pair = sc.AnnData(pla_obj.protein_pair_count.transpose().to_numpy(), obsm={'spatial': tissue_barcode[['pxl_col_in_fullres','pxl_row_in_fullres']].to_numpy()}, obs=tissue_barcode[['in_tissue','array_row','array_col']])


In [ ]:
#rename var name
adata_pla.var.index = pla.index
adata_protein.var.index = pla_obj.protein_count.index
adata_protein_pair.var.index = pla_obj.protein_pair_count.index


In [ ]:
#read image 
image = mpimg.imread('D1_outs/spatial/tissue_hires_image.png')
def set_uns_parameters(adata, library_id, image):
    spatial_key = "spatial"
    adata.uns[spatial_key] = {library_id: {"images": {"hires": image},
                                           "scalefactors": {'tissue_hires_scalef': 0.53590566,
                                                            'tissue_lowres_scalef': 0.1607717,
                                                            'fiducial_diameter_fullres': 45.420517,
                                                            'spot_diameter_fullres': 28.117462}}}

# Assuming adata_rna, adata_protein, and adata_protein_pair are your AnnData objects
library_id = "D1_Human_tonsil_Proxseq"
image = image  # Replace with your actual image variable

set_uns_parameters(adata_pla, library_id, image)
set_uns_parameters(adata_protein, library_id, image)
set_uns_parameters(adata_protein_pair, library_id, image)


In [ ]:
#plot it and you can get similar result as RNA 
adata_protein_pair.obs['PLA'] = 'PLA'
# plt.rcParams["figure.figsize"] = (8, 8)
#sc.pl.spatial(adata_protein_pair, color = 'PLA')
adata_protein_pair.var['IgG'] = adata_protein_pair.var_names.str.contains('IgG1a')
adata_protein_pair.var['feature_types'] = 'protein_pair'
sc.pp.calculate_qc_metrics(
    adata_protein_pair,
    percent_top=(5, 10, 15),
    var_type="protein_pair",
    qc_vars=("IgG",),
    inplace=True,
)

In [ ]:
sc.pl.spatial(adata_protein_pair, color = ['total_counts','n_protein_pair_by_counts'],  size = 1.5, save = 'Extended-Fig-2g.svg')

In [ ]:
# For PLA data:
mean_counts = np.mean(adata_protein_pair.obs['total_counts'])
median_counts = np.median(adata_protein_pair.obs['total_counts'])
std_counts = np.std(adata_protein_pair.obs['total_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")

# For PLA data:
mean_counts = np.mean(adata_protein_pair.obs['n_protein_pair_by_counts'])
median_counts = np.median(adata_protein_pair.obs['n_protein_pair_by_counts'])
std_counts = np.std(adata_protein_pair.obs['n_protein_pair_by_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")

In [ ]:
#keep index both in rna and pla
obs_names = adata_protein_pair.obs.index.intersection(adata.obs.index)
adata_protein_pair = adata_protein_pair[adata_protein_pair.obs.index.isin(list(obs_names))]
adata = adata[adata.obs.index.isin(list(obs_names))]

In [ ]:
#similar preprocess to PLA data
adata_protein_pair.layers["counts"] = adata_protein_pair.X.copy() # preserve counts
sc.pp.normalize_total(adata_protein_pair, target_sum=1e4) # scale each cell to a common library size
sc.pp.log1p(adata_protein_pair) # log(expression + 1)
adata_protein_pair.raw = adata_protein_pair.copy() # freeze the state in `.raw`
adata_protein_pair.write("adata_D1_protein_pair_raw_data.h5ad")
sc.pp.scale(adata_protein_pair, max_value=10)

In [ ]:
sc.tl.pca(adata_protein_pair, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata_protein_pair, log=True)

In [ ]:
sc.pp.neighbors(adata_protein_pair, n_neighbors=30, n_pcs=20)
sc.tl.umap(adata_protein_pair)


In [ ]:
sc.tl.leiden(adata_protein_pair,key_added='leiden_pla',resolution=0.7)
sc.pl.spatial(adata_protein_pair,color='leiden_pla', save = 'Extended-Fig-4f-2.svg')

In [ ]:
sc.pl.umap(adata_protein_pair,color='leiden_pla', save = 'Extended-Fig-4f-1.svg')

In [ ]:
sc.tl.rank_genes_groups(adata_protein_pair, groupby='leiden_pla', method='wilcoxon')
sc.pl.rank_genes_groups(adata_protein_pair, groupby='leiden_pla', method='wilcoxon')

In [ ]:
sc.tl.dendrogram(adata_protein_pair, groupby='leiden_pla', key_added='dendrogram_annotation')

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata_protein_pair, n_genes=4, groupby='leiden_pla', swap_axes=True, use_raw=False, vmin=-4, vmax=4,
                                cmap='bwr', figsize=(10, 10))

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_protein_pair, n_genes=3)

In [ ]:
adata_protein_pair.obs['mrna_annotation'] = adata.obs['mrna_annotation']

In [ ]:
sc.pl.umap(adata_protein_pair,color='mrna_annotation')

In [ ]:
sc.pl.umap(adata_protein_pair,color='leiden_pla')

In [ ]:
sc.tl.dendrogram(adata_protein_pair, groupby='mrna_annotation', key_added='dendrogram_annotation')

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata_protein_pair, n_genes=5, groupby='mrna_annotation', swap_axes=True, use_raw=False, vmin=-4, vmax=4,
                                cmap='bwr', figsize=(10, 14))

In [ ]:
#plot it and you can get similar result as RNA 
adata_protein = adata_protein[adata_protein.obs.index.isin(list(obs_names))]
adata_protein.obs['protein'] = 'protein'
# plt.rcParams["figure.figsize"] = (8, 8)
#sc.pl.spatial(adata_protein, color = 'protein')
adata_protein.var['IgG'] = adata_protein.var_names.str.contains('IgG1a')
adata_protein.var['feature_types'] = 'protein'
sc.pp.calculate_qc_metrics(
    adata_protein,
    percent_top=(5, 10, 15),
    var_type="protein",
    qc_vars=("IgG",),
    inplace=True,
)

In [ ]:
sc.pl.spatial(adata_protein, color = ['total_counts','n_protein_by_counts'],  size = 1.5, save = 'Extended-Fig-2h.svg')

In [ ]:
# For protein data:
mean_counts = np.mean(adata_protein.obs['total_counts'])
median_counts = np.median(adata_protein.obs['total_counts'])
std_counts = np.std(adata_protein.obs['total_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")

# For protein data:
mean_counts = np.mean(adata_protein.obs['n_protein_by_counts'])
median_counts = np.median(adata_protein.obs['n_protein_by_counts'])
std_counts = np.std(adata_protein.obs['n_protein_by_counts'])

print(f"Mean: {mean_counts:.1f}, Median: {median_counts:.1f}, Std: {std_counts:.1f}")


In [ ]:
adata_protein.layers["counts"] = adata_protein.X.copy() # preserve counts
sc.pp.normalize_total(adata_protein, target_sum=1e4) # scale each cell to a common library size
sc.pp.log1p(adata_protein) # log(expression + 1)
adata_protein.raw = adata_protein.copy() # freeze the state in `.raw`
adata_protein.write("adata_D1_protein_raw_data.h5ad")
sc.pp.scale(adata_protein, max_value=10)
sc.tl.pca(adata_protein, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata_protein, log=True)

In [ ]:
sc.pp.neighbors(adata_protein, n_neighbors=20, n_pcs=23)
sc.tl.umap(adata_protein)

In [ ]:
sc.tl.leiden(adata_protein,key_added='leiden_protein',resolution=0.7)
sc.pl.spatial(adata_protein, color = ['leiden_protein'], save = 'Extended-Fig-4e-2.svg')

In [ ]:
sc.pl.umap(adata_protein, color = ['leiden_protein'], save = 'Extended-Fig-4e-1.svg')

In [ ]:
# Copy cluster annotation from `adata` to `adata_pearson`
adata_protein.obs['mrna_annotation'] = adata.obs['mrna_annotation']


In [ ]:
sc.tl.rank_genes_groups(adata_protein, groupby='mrna_annotation', method='wilcoxon')
sc.pl.rank_genes_groups(adata_protein, groupby='mrna_annotation', method='wilcoxon')

In [ ]:
sc.tl.dendrogram(adata_protein, groupby='mrna_annotation', key_added='dendrogram_annotation')


In [ ]:
sc.pl.rank_genes_groups_heatmap(
    adata_protein,
    groupby='mrna_annotation',
    swap_axes=True,
    cmap='bwr',
    use_raw=False,
    n_genes = 4,
    vmin=-5,
    vmax=5,
    figsize=(8,10)
)


In [ ]:
custom_protein_order = [ 'CD35', 'VCAM1',  'IgM','HIgG', 'CD86','PDL2', 'CD21', 'CD23', 'IgD', 'CD20', 'CD19',  'CD40', 'ITGA4', 'CD24', 'LFA1','CD11a',
    'CD29','CD81','CD38', 'CD8',  'CD3',  'CD4','CD27','CD279','CD28', 'CD32','CD9', 'CD147','ICAM1',]

# Plot the heatmap using your custom protein order
sc.pl.rank_genes_groups_heatmap(
    adata_protein,
    groupby='mrna_annotation',
    var_names=custom_protein_order,  # Use your custom order here
    swap_axes=True,
    cmap='RdBu_r',
    use_raw=False,
    vmin=-3.5,
    vmax=3.5,
    save='Extended-Fig-6b.svg',
    figsize=(7, 7)
)


In [ ]:
sc.pl.umap(adata_protein,color='leiden_protein')

In [ ]:
sc.pl.umap(adata_protein,color='mrna_annotation')

In [ ]:
fraction_overlap = pd.read_csv('D1_weighted_sc_pla.csv', index_col=0)
adata_fraction = sc.AnnData(fraction_overlap.transpose().to_numpy(), obsm={'spatial': tissue_barcode[['pxl_col_in_fullres','pxl_row_in_fullres']].to_numpy()}, obs=tissue_barcode[['in_tissue','array_row','array_col']])
adata_fraction.var.index = fraction_overlap.index
set_uns_parameters(adata_fraction, library_id, image)
#keep index both in rna and pla
obs_names = adata_fraction.obs.index.intersection(adata.obs.index)
adata_fraction = adata_fraction[adata_fraction.obs.index.isin(list(obs_names))]
#similar preprocess to PLA data
adata_fraction.layers["counts"] = adata_fraction.X.copy() # preserve counts
# sc.pp.normalize_total(adata_w, target_sum=1e4) # scale each cell to a common library size
# sc.pp.log1p(adata_w) #log(expression + 1)
adata_fraction.raw = adata_fraction.copy() # freeze the state in `.raw`
sc.pp.scale(adata_fraction, max_value=10)
sc.tl.pca(adata_fraction, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_fraction, log=True)
sc.pp.neighbors(adata_fraction, n_neighbors=10, n_pcs=6)
sc.tl.umap(adata_fraction)
sc.tl.leiden(adata_fraction,key_added='Fractional_overlap',resolution=0.5)
sc.pl.spatial(adata_fraction, color = ['Fractional_overlap'], save='Extended-Fig-10c.svg')


In [ ]:
sc.pl.spatial(adata_protein, color = ['CD29', ], cmap='coolwarm', spot_size=45,)

In [ ]:
sc.pl.spatial(adata_protein, color = ['ITGA4', ], cmap='coolwarm', spot_size=45,save = 'Fig-6c-1.svg')

In [ ]:
sc.pl.spatial(adata_protein, color = ['VCAM1', ], cmap='coolwarm', spot_size=45,save = 'Fig-6c-2.svg')

In [ ]:
sc.pl.spatial(adata_protein_pair, color = ['ITGA4:VCAM1' ], layer = 'counts', cmap='coolwarm', spot_size=45,vmin = 0, vmax = 45, save = 'Fig-6c-3.svg')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import mannwhitneyu
import os

# Extract data
adata = adata_protein_pair
pair = 'ITGA4:VCAM1'

# Get raw counts from layers
counts = adata[:, pair].layers["counts"].toarray().flatten()
zones = adata.obs["mrna_annotation"]

# Build DataFrame and filter only Light zone and Dark zone
df = pd.DataFrame({
    "ITGA4:VCAM1": counts,
    "Zone": zones
})
df = df[df["Zone"].isin(["Dark zone", "Light zone"])]

# Define order and color palette
order = ["Dark zone", "Light zone"]
palette = {
    "Dark zone": "#66c2a5",
    "Light zone": "#fc8d62"
}

# Perform statistical test
group1 = df[df["Zone"] == "Dark zone"]["ITGA4:VCAM1"]
group2 = df[df["Zone"] == "Light zone"]["ITGA4:VCAM1"]
stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
print(f"🔍 Mann-Whitney U test p-value: {p_value:.4g}")

# Determine significance stars
if p_value < 0.001:
    stars = '***'
elif p_value < 0.01:
    stars = '**'
elif p_value < 0.05:
    stars = '*'
else:
    stars = 'ns'

# Plot
plt.figure(figsize=(3.5, 3))
ax = sns.boxplot(
    data=df,
    x="Zone",
    y="ITGA4:VCAM1",
    order=order,
    palette=palette,
    showcaps=True,
    boxprops=dict(linewidth=0),  # ⬅️ Remove box border
    whiskerprops=dict(linewidth=1),
    medianprops=dict(linewidth=1, color='black')
)
sns.stripplot(data=df, x="Zone", y="ITGA4:VCAM1", color="black", size=2, jitter=True, alpha=0.3, order=order)

# Draw significance bar
y_max = df["ITGA4:VCAM1"].max()
y1 = y_max * 0.95
y2 = y_max * 0.95
x1, x2 = 0, 1
ax.plot([x1, x1, x2, x2], [y1, y2, y2, y1], lw=1.2, color='black')
ax.text((x1 + x2) / 2, y2 - 0.5, stars, ha='center', va='bottom', fontsize=10)

# Labels and layout
plt.ylabel("Raw counts")
plt.xlabel("")
plt.title("ITGA4:VCAM1")
plt.tight_layout()

# Save figure
os.makedirs("Figures", exist_ok=True)
plt.savefig("Fig-6d-3.png", dpi=300)
plt.savefig("Fig-6d-3.svg", dpi=300)

plt.show()
